# Dialogue and Narrative Coursework - Subtask 1

In [ ]:
## imports
%%capture
!pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
# Link Google Drive to get new dataset
from google.colab import drive
drive.mount('/gdrive')

# %cd drive/MyDrive/Colab\ Notebooks/Dialogue\ &\ Narrative
%cd ~
%cd /gdrive/My\ Drive
%cd Colab\ Notebooks/Dialogue\ &\ Narrative
print()

%ls .

Mounted at /gdrive
/root
/gdrive/My Drive
/gdrive/My Drive/Colab Notebooks/Dialogue & Narrative

 data_cache/                predictions_subtask1.json  'Week 3'/  'Week 8'/
 doc2dial_rc_train.csv      test_doc2vec.model         'Week 4'/  'Week 9'/
 doc2dial_rc_val.csv       'Week 1'/                   'Week 5'/
'Doc2Dial - Task 1.ipynb'  'Week 2'/                   'Week 7'/


## Load the data

In [ ]:
## A function to load a specific doc2dial dataset

def load_doc2dial_dataset(name='dialogue_domain', split='train'):
  cache_dir = "./data_cache"

  return load_dataset(
      "doc2dial",
      name=name,
      split=split,
      ignore_verifications=True,
      cache_dir=cache_dir,
  )


## Understanding an example

In [ ]:
## Load documents 

documents = load_doc2dial_dataset(
              name="document_domain",
              split="train",
            )

Downloading:   0%|          | 0.00/3.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
documents[0]

{'doc_html_raw': '<main class="content" id="content" role="main">\n\n<section>\n\n<div>\n<h2>Benefits Planner: Survivors | Planning For Your Survivors</h2>\n</div>\n</section>\n\n<section>\n\n<div>\n\n<div>\n\n\n</div>\n\n\n\n<article>\n<section>\n<p>As you plan for the future, you\'ll want to think about what your family would need if you should die now. Social Security can help your family if you have earned enough Social Security credits through your work.</p>\n<p><a>You can earn up to four credits each year</a>. In 2019, for example, you earn one credit for each $1,360 of wages or <a>self-employment</a> income. When you have earned $5,440, you have earned your four credits for the year.</p>\n<p>The number of credits needed to provide benefits for your survivors depends on your age when you die. No one needs more than 40 credits (10 years of work) to be eligible for any Social Security benefit. But, the younger a person is, the fewer credits they must have for family members to rece

In [ ]:
## Load dialogue turns

dialogues = load_doc2dial_dataset(
              name="dialogue_domain",
              split="validation",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
dialogues[0]

{'dial_id': 'dea7174409afbfe0af0ace21e7f318ae',
 'doc_id': 'Top 5 DMV Mistakes and How to Avoid Them#3_0',
 'domain': 'dmv',
 'turns': [{'da': 'query_condition',
   'references': [{'label': 'precondition', 'sp_id': '23'}],
   'role': 'user',
   'turn_id': 1,
   'utterance': 'My insurance ended so what should i do'},
  {'da': 'respond_solution',
   'references': [{'label': 'solution', 'sp_id': '24'},
    {'label': 'solution', 'sp_id': '25'},
    {'label': 'solution', 'sp_id': '26'}],
   'role': 'agent',
   'turn_id': 2,
   'utterance': 'You will need to get insurance or we will suspend your registration and license'},
  {'da': 'query_condition',
   'references': [{'label': 'precondition', 'sp_id': '28'}],
   'role': 'user',
   'turn_id': 3,
   'utterance': "Don't do that I'll get insurance"},
  {'da': 'respond_solution',
   'references': [{'label': 'solution', 'sp_id': '29'}],
   'role': 'agent',
   'turn_id': 4,
   'utterance': 'Okay, have you received a letter from the DMV letting you

For subtask 1, the gound truths are specified in the reading comprehension dataset 'doc2dial_rc'. Let's show this on the first example from the validation set.

In [ ]:
## Using the doc2dial_rc dataset (also used for evaluation)

qas = load_doc2dial_dataset(
              name="doc2dial_rc",
              split="validation",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
## The following display will be used as ground truth for evaluation
ref = {'id':qas[0]['id'], 'answers':qas[0]['answers']}
ref

{'answers': {'answer_start': [1796],
  'text': ['Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , ']},
 'id': 'dea7174409afbfe0af0ace21e7f318ae_1'}

The answer corresponding to question `dea7174409afbfe0af0ace21e7f318ae_1` most likely has id `dea7174409afbfe0af0ace21e7f318ae_2`. So let's check it using the given dialogue and document domains datasets. We should also check that this answer corresponds to a span starting at `1796` in the associated document.

In [ ]:
## Let's find anwer 'dea7174409afbfe0af0ace21e7f318ae_2'

question_id = 'dea7174409afbfe0af0ace21e7f318ae_2'
dial = 'dea7174409afbfe0af0ace21e7f318ae'
turn = 2

for ex in dialogues:
  if ex['dial_id'] == dial:

    doc_id = ex['doc_id']   ## corresponding document id
    span_ids = []           ## corresponding span ids

    for tr in ex['turns']:
      if tr['turn_id'] == turn:
        print(tr)

        for sp in tr['references']:
          span_ids.append(sp['sp_id'])

{'turn_id': 2, 'role': 'agent', 'da': 'respond_solution', 'references': [{'sp_id': '24', 'label': 'solution'}, {'sp_id': '25', 'label': 'solution'}, {'sp_id': '26', 'label': 'solution'}], 'utterance': 'You will need to get insurance or we will suspend your registration and license'}


In [ ]:
## Let's get the correpond answer in the document_domain

for doc in documents:
  if doc['doc_id'] == doc_id:

    ans = ''
    for span in doc['spans']:
      if span['id_sp'] in span_ids:
        ans += span['text_sp']

print(ans)

Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , 


In [ ]:
## Let's check that the effectively starts at 1796

for doc in documents:
  if doc['doc_id'] == doc_id:
    for span in doc['spans']:
      if span['start_sp'] == 1796:
        print(span['text_sp'])

Because we all pay indirectly for crashes involving uninsured motorists , 


It checks out! Now that we know that our ground truths are essentially doc2dial_rc, let see what our predictions should look like in order to comply with the evaluation script.

In [ ]:
## Examples of predictions
[
    {
        "id": "dea7174409afbfe0af0ace21e7f318ae_1",
        "prediction_text": "Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage ,",
        "no_answer_probability": 0.883711576461792
    },
    {
        "id": "dea7174409afbfe0af0ace21e7f318ae_9",
        "prediction_text": "we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. ",
        "no_answer_probability": 0.0004379140445962548
    }
]

[{'id': 'dea7174409afbfe0af0ace21e7f318ae_1',
  'no_answer_probability': 0.883711576461792,
  'prediction_text': 'Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage ,'},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_9',
  'no_answer_probability': 0.0004379140445962548,
  'prediction_text': 'we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. '}]

## Simple model based on a cosine similarity

### Build new datasets (much more simple ones)

We know our ouputs and their ground thruths, we must now build our inputs. 

In [ ]:
train_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="train")
val_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="validation")

documents = load_doc2dial_dataset(name="document_domain", split="train")

train_data = load_doc2dial_dataset(name="doc2dial_rc", split="train")
val_data = load_doc2dial_dataset(name="doc2dial_rc", split="validation")

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)
Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)
Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)
Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)
Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
train_data[0]

{'answers': {'answer_start': [346],
  'text': ['you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ']},
 'context': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the poli

In [ ]:
def add_spans(sample, dialogues, documents):
  dial_id, turn_id = sample['id'].split('_')

  dials = dialogues.filter(lambda ex: ex['dial_id'] == dial_id)[0]

  doc_id = dial['doc_id']   ## corresponding document id
  span_ids = []             ## corresponding span ids

  for tr in dial['turns']:
    if tr['turn_id'] == int(turn_id)+1:

      for sp in tr['references']:
        span_ids.append(sp['sp_id'])

      break

  doc = documents.filter(lambda ex: ex['doc_id'] == doc_id)[0]
  
  spans = {}
  answer_spans = {}
  for span in doc['spans']:
    spans[span['id_sp']] = span['text_sp']
    if span['id_sp'] in span_ids:
      answer_spans[span['id_sp']] = span['text_sp']

  sample['spans'] = spans
  sample['answers']['spans'] = answer_spans

  return sample

In [ ]:
# new_data_train = data_train.map(add_spans)

# new_data_train[0]

In [ ]:
# %%capture
# from tqdm import tqdm

# new_train_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])

# for ex in tqdm(train_data):
#   new_train_data = new_train_data.append(add_spans(ex, train_dialogues), ignore_index=True)

# new_train_data.head()

In [ ]:
# new_train_data.to_csv('doc2dial_rc_train.csv')
new_train_data = pd.read_csv('doc2dial_rc_train.csv')
new_train_data.head()

,Unnamed: 0,id,question,context,answers,spans,domain,title
0,0,9f44c1539efe6f7e79b02eb1b413aa43_1,"user:Hello, I forgot o update my address, can ...",Many DMV customers make easily avoidable mista...,{'text': ['you must report a change of address...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
1,1,9f44c1539efe6f7e79b02eb1b413aa43_3,user:Can I do my DMV transactions online? \tag...,Many DMV customers make easily avoidable mista...,"{'text': ['Sign up or log into MyDMV [6 ] '], ...",{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
2,2,9f44c1539efe6f7e79b02eb1b413aa43_5,"user:Thanks, and in case I forget to bring all...",Many DMV customers make easily avoidable mista...,{'text': ['About ten percent of customers visi...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
3,3,9f44c1539efe6f7e79b02eb1b413aa43_7,"user:Ok, and can you tell me again where shoul...",Many DMV customers make easily avoidable mista...,{'text': ['you must report a change of address...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
4,4,9f44c1539efe6f7e79b02eb1b413aa43_9,user:Can you tell me more about Traffic points...,Many DMV customers make easily avoidable mista...,{'text': ['DMV maintains a point system to tra...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0


In [ ]:
new_train_data

,Unnamed: 0,id,question,context,answers,spans,domain,title
0,0,9f44c1539efe6f7e79b02eb1b413aa43_1,"user:Hello, I forgot o update my address, can ...",Many DMV customers make easily avoidable mista...,{'text': ['you must report a change of address...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
1,1,9f44c1539efe6f7e79b02eb1b413aa43_3,user:Can I do my DMV transactions online? \tag...,Many DMV customers make easily avoidable mista...,"{'text': ['Sign up or log into MyDMV [6 ] '], ...",{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
2,2,9f44c1539efe6f7e79b02eb1b413aa43_5,"user:Thanks, and in case I forget to bring all...",Many DMV customers make easily avoidable mista...,{'text': ['About ten percent of customers visi...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
3,3,9f44c1539efe6f7e79b02eb1b413aa43_7,"user:Ok, and can you tell me again where shoul...",Many DMV customers make easily avoidable mista...,{'text': ['you must report a change of address...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
4,4,9f44c1539efe6f7e79b02eb1b413aa43_9,user:Can you tell me more about Traffic points...,Many DMV customers make easily avoidable mista...,{'text': ['DMV maintains a point system to tra...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
...,...,...,...,...,...,...,...,...
545,545,5ad608ae0081e4652a687943276aca6b_5,user:Do i need to pay any fee to appeal? \tage...,\n\nAppeal a TVB ticket conviction[1] \n\n\nYo...,{'text': ['you must file your appeal within 30...,{'1': '\n\nAppeal a TVB ticket conviction[1] \...,dmv,Appeal a TVB ticket conviction#1_0
546,546,5ad608ae0081e4652a687943276aca6b_7,user:How to appeal it by mail? \tagent:Yes. A ...,\n\nAppeal a TVB ticket conviction[1] \n\n\nYo...,"{'text': ['Mail your appeal and fee to '], 'an...",{'1': '\n\nAppeal a TVB ticket conviction[1] \...,dmv,Appeal a TVB ticket conviction#1_0
547,547,5ad608ae0081e4652a687943276aca6b_9,user:Can i submit an additional Appeal Arguemn...,\n\nAppeal a TVB ticket conviction[1] \n\n\nYo...,{'text': ['you may submit an additional Appeal...,{'1': '\n\nAppeal a TVB ticket conviction[1] \...,dmv,Appeal a TVB ticket conviction#1_0
548,548,5ad608ae0081e4652a687943276aca6b_11,user:Can NYS law requirements affect my appeal...,\n\nAppeal a TVB ticket conviction[1] \n\n\nYo...,{'text': ['The DMV Appeals Board will reject y...,{'1': '\n\nAppeal a TVB ticket conviction[1] \...,dmv,Appeal a TVB ticket conviction#1_0


In [ ]:
# %%capture
# new_val_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])

# for ex in tqdm(val_data):
#   new_val_data = new_val_data.append(add_spans(ex, val_dialogues), ignore_index=True)

# new_val_data.head()

In [ ]:
# new_val_data.to_csv('doc2dial_rc_val.csv')
new_val_data = pd.read_csv('doc2dial_rc_val.csv')
new_val_data.head()

,Unnamed: 0,id,question,context,answers,spans,domain,title
0,0,dea7174409afbfe0af0ace21e7f318ae_1,user:My insurance ended so what should i do,Many DMV customers make easily avoidable mista...,{'text': ['Because we all pay indirectly for c...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
1,1,dea7174409afbfe0af0ace21e7f318ae_3,user:Don't do that I'll get insurance \tagent:...,Many DMV customers make easily avoidable mista...,{'text': ['we mail you an insurance inquiry le...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
2,2,dea7174409afbfe0af0ace21e7f318ae_5,"user:I have, that is why I am here to clear th...",Many DMV customers make easily avoidable mista...,{'text': ['Learn more about how to change the ...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
3,3,dea7174409afbfe0af0ace21e7f318ae_7,user:Will I still have to turn in anything or ...,Many DMV customers make easily avoidable mista...,{'text': ['you won t learn that you have an in...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0
4,4,dea7174409afbfe0af0ace21e7f318ae_9,"user:OKay, well I just logged in and I have in...",Many DMV customers make easily avoidable mista...,{'text': ['Because we all pay indirectly for c...,{'1': 'Many DMV customers make easily avoidabl...,dmv,Top 5 DMV Mistakes and How to Avoid Them#3_0


### Test how to use a doc2vec model 

In [ ]:
# import ast
# new_val = new_val_data.apply(lambda row : ast.literal_eval(row), axis=1)

# new_val.head()

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import ast

index = 0
instance = new_train_data.iloc[index]
# print((instance['spans']))
docs = ast.literal_eval(instance['spans'])
print(docs)

# Tokenization of each document
tokenized_docs = {}
for i, d in docs.items():
    tokenized_docs[int(i)] = word_tokenize(d.lower())
print(tokenized_docs)

# Convert tokenized document into gensim formated tagged data
tagged_data = []
for i, d in tokenized_docs.items():
  tagged_data.append(TaggedDocument(d, [i]))

tagged_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
{'1': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. ', '2': 'Because we see customers make these mistakes over and over again , ', '3': 'we are issuing this list of the top five DMV mistakes and how to avoid them. ', '4': '\n\n1. Forgetting to Update Address \n', '5': 'By statute , ', '6': 'you must report a change of address to DMV within ten days of moving. ', '7': 'That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ', '8': 'It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. ', '9': 'If you fail to keep your address current , ', '10': 'you will miss a 

[TaggedDocument(words=['many', 'dmv', 'customers', 'make', 'easily', 'avoidable', 'mistakes', 'that', 'cause', 'them', 'significant', 'problems', ',', 'including', 'encounters', 'with', 'law', 'enforcement', 'and', 'impounded', 'vehicles', '.'], tags=[1]),
 TaggedDocument(words=['because', 'we', 'see', 'customers', 'make', 'these', 'mistakes', 'over', 'and', 'over', 'again', ','], tags=[2]),
 TaggedDocument(words=['we', 'are', 'issuing', 'this', 'list', 'of', 'the', 'top', 'five', 'dmv', 'mistakes', 'and', 'how', 'to', 'avoid', 'them', '.'], tags=[3]),
 TaggedDocument(words=['1.', 'forgetting', 'to', 'update', 'address'], tags=[4]),
 TaggedDocument(words=['by', 'statute', ','], tags=[5]),
 TaggedDocument(words=['you', 'must', 'report', 'a', 'change', 'of', 'address', 'to', 'dmv', 'within', 'ten', 'days', 'of', 'moving', '.'], tags=[6]),
 TaggedDocument(words=['that', 'is', 'the', 'case', 'for', 'the', 'address', 'associated', 'with', 'your', 'license', ',', 'as', 'well', 'as', 'all', '

In [ ]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)

## Print model vocabulary
# model.wv.vocab

In [ ]:
# find most similar doc 
# test_doc = ast.literal_eval(instance['answers'])['text'][0]
test_doc = instance['question'][5:]
print(test_doc)
print()
print(ast.literal_eval(instance['answers'])['spans'].keys())
predictions  = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)], topn=30)
predictions

Hello, I forgot o update my address, can you help me with that?

dict_keys(['6', '7'])


[(52, 0.999070405960083),
 (54, 0.9951107501983643),
 (20, 0.9947805404663086),
 (16, 0.9947336316108704),
 (37, 0.9947190284729004),
 (17, 0.9946351051330566),
 (6, 0.9944206476211548),
 (35, 0.9942614436149597),
 (18, 0.994206964969635),
 (14, 0.9942036867141724),
 (27, 0.9941309690475464),
 (25, 0.994062066078186),
 (24, 0.9939948320388794),
 (50, 0.9939670562744141),
 (11, 0.9935005307197571),
 (10, 0.9933738708496094),
 (4, 0.9930968880653381),
 (7, 0.9930293560028076),
 (53, 0.9929778575897217),
 (48, 0.992828369140625),
 (30, 0.9928181767463684),
 (55, 0.9927996397018433),
 (56, 0.9927851557731628),
 (33, 0.992763876914978),
 (12, 0.992724597454071),
 (9, 0.9924390316009521),
 (38, 0.9923851490020752),
 (41, 0.9923475980758667),
 (44, 0.9923113584518433),
 (46, 0.9923081994056702)]

In [ ]:
## Generate predictions in the right format

text = ''
count = 0
sum_score = 0
pred = 0
while True:

  id, score = predictions[pred]
  text += docs[str(id)]
  count += 1
  sum_score += score

  pred += 1
  next_id, next_score = predictions[pred]
  if next_id != id+1:
    break

final_score = sum_score / count

print('Question:', instance['question'][5:])
print('--------------------------------\n')
print('Correct answer:   ', ast.literal_eval(instance['answers'])['text'][0])
print()
print('Our answer:       ', text)

Question: Hello, I forgot o update my address, can you help me with that?
--------------------------------

Correct answer:    you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. 

Our answer:        don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. 


In [ ]:
print('Anwswer in required format:\n')
final_prediction = {"id":instance["id"], "prediction_text":text, "no_answer_probability":1.0-final_score}
final_prediction

Anwswer in required format:



{'id': '9f44c1539efe6f7e79b02eb1b413aa43_1',
 'no_answer_probability': 0.0009295940399169922,
 'prediction_text': 'don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. '}

### Wrap everything and make predictions on the val set

In [ ]:
def train_predict_doc2vec(dataset, index):
  instance = dataset.iloc[index]
  docs = ast.literal_eval(instance['spans'])

  tokenized_docs = {}
  for i, d in docs.items():
      tokenized_docs[int(i)] = word_tokenize(d.lower())
      # if int(i)==0:
      #   print("gotcha")

  tagged_data = []
  for i, d in tokenized_docs.items():
    tagged_data.append(TaggedDocument(d, [i]))
    # if i==0:
    #   print("gotcha")

  model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)

  test_doc = instance['question'][5:]
  predictions = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)], topn=30)
  # print(predictions)

  text = ''
  count = 0
  sum_score = 0
  pred = 0
  while True:

    id, score = predictions[pred]
    text += docs.get(str(id), '')
    count += 1
    sum_score += score

    pred += 1
    next_id, next_score = predictions[pred]
    if next_id != id+1:
      break

  final_score = sum_score / count
  return {"id":instance["id"], "prediction_text":text, "no_answer_probability":1.0-final_score}

In [ ]:
from tqdm import tqdm

preds = []
new_val_data_slice = new_val_data[:100]

for i in tqdm(range(len(new_val_data_slice))):
  preds.append(train_predict_doc2vec(new_val_data_slice, i))

preds

100%|██████████| 100/100 [01:04<00:00,  1.55it/s]


[{'id': 'dea7174409afbfe0af0ace21e7f318ae_1',
  'no_answer_probability': 0.00047957897186279297,
  'prediction_text': 'Make sure you turn in your vehicle s license plates at DMV before you cancel your insurance policy. '},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_3',
  'no_answer_probability': 0.0033761262893676758,
  'prediction_text': 'don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. '},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_5',
  'no_answer_probability': 0.045258522033691406,
  'prediction_text': 'don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. '},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_7',
  'no_answer_probability': 0.09994685649871826,
  'prediction_

In [ ]:
import json

with open('predictions_subtask1.json', 'w') as outfile:
    json.dump(preds, outfile)
